In [1]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [2]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

In [3]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# create the model with transe scoring function
model = ScoringBasedEmbeddingModel(eta=1, 
                                     k=10,
                                     scoring_type='TransE')


# compile the model with loss and optimizer
model.compile(optimizer='adam', loss='multiclass_nll')

# Use this for checkpoints at regular intervals
#checkpoint = tf.keras.callbacks.ModelCheckpoint('./chkpt_transe', monitor='val_mrr', verbose=1, save_best_only=True, mode='min')

# Use this for early stopping
checkpoint = tf.keras.callbacks.EarlyStopping(monitor="val_mrr", # which metrics to monitor
                                              patience=3,        # If the monitored metric doesnt improve 
                                                                 # for these many checks the model early stops
                                              verbose=1,         # verbosity
                                              mode="max",        # how to compare the monitored metrics. 
                                                                 # max - means higher is better
                                              restore_best_weights=True) # restore the weights with best value

dataset = load_fb15k_237()

model.fit(dataset['train'],
             batch_size=10000,
             epochs=100,
             validation_freq=2,
             validation_batch_size=100,
             validation_data = dataset['valid'][::100],
         callbacks=[checkpoint])     # Pass the callback to the fit function


Epoch 1/100
29/29 [==============================] - 2s 76ms/step - loss: 6698.2188
Epoch 2/100
29/29 [==============================] - 2s 70ms/step - loss: 6648.8862 - val_mrr: 0.0672 - val_mr: 2604.4943 - val_hits@1: 0.0000e+00 - val_hits@10: 0.1847
Epoch 3/100
29/29 [==============================] - 1s 33ms/step - loss: 6590.2842
Epoch 4/100
29/29 [==============================] - 1s 43ms/step - loss: 6517.4517 - val_mrr: 0.0918 - val_mr: 1316.6307 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2528
Epoch 5/100
29/29 [==============================] - 1s 32ms/step - loss: 6431.8696
Epoch 6/100
29/29 [==============================] - 1s 43ms/step - loss: 6338.9854 - val_mrr: 0.0873 - val_mr: 935.4460 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2443
Epoch 7/100
29/29 [==============================] - 1s 32ms/step - loss: 6243.9902
Epoch 8/100
29/29 [==============================] - 2s 69ms/step - loss: 6148.5376 - val_mrr: 0.0843 - val_mr: 857.0710 - val_hits@1: 0.0000e+00 - val_hits@1

In [4]:
# evaluate on the test set
ranks = model.evaluate(dataset['test'], # test set
                       batch_size=100, # evaluation batch size
                       corrupt_side='s,o', 
                       use_filter={'train':dataset['train'], # Filter to be used for evaluation
                                   'valid':dataset['valid'],
                                   'test':dataset['test']}
                       )

# import the evaluation metrics
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score

print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('hits@1:', hits_at_n_score(ranks, 1))
print('hits@10:', hits_at_n_score(ranks, 10))

206/206 [==============================] - 85s 413ms/step
MR: 1181.2051325961445
MRR: 0.18521472801873523
hits@1: 0.1281191897445934
hits@10: 0.29547900968783636


In [ ]:
# if using ModelCheckpoint then we can restore the checkpoints using restore model
# from ampligraph.utils import restore_model
# model = restore_model('chkpt_transe')